# Exception Rates for New Scarlet

Count up the exception rates from the logs. 

## Data

Used the run in:
`/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym/logs`

This was produced with the following versions:
```
lsst_distrib 17.0+52      current w_2019_19 setup

autograd              LOCAL:/project/yusra/lsst_devel/autograd/_eupspkg/binary/autograd/master-g6a9ef8c4bf      setup
meas_extensions_scarlet LOCAL:/project/yusra/lsst_devel/meas_extensions_scarlet         setup
pipe_drivers          LOCAL:/project/yusra/lsst_devel/pipe_drivers      setup
pipe_tasks            LOCAL:/project/yusra/lsst_devel/pipe_tasks        setup
scarlet               LOCAL:/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3    setup
```

- *where pipe_tasks has is on DM-19720:*
```
commit aa5e78b53f2bac60691a206f0688554dfb74b06c
Author: Nate Lust <nlust@astro.princeton.edu>
Date:   Mon May 13 08:57:20 2019 -0500

    Fix deblender logging
```


- *pipe_drivers is on DM-19720:*
```
commit f880519c69f5ebe1ae0d6632a95167e579490c9b
Author: Yusra AlSayyad <yusra@astro.princeton.edu>
Date:   Mon May 13 15:58:24 2019 -0500

    Do not check for
    
    which is not a config parameter in ScarletDeblendTask
```


- *meas_extensions_scarlet is on DM-19720 with a cherry-pick of Fred’s latest commit `git cherry-pick 136e4054f24572e1ba3d1566f5cfbe85c1218b53` on `deblender_sprint`. The sha changed because it required a merge-fix:*

```
commit 9c6da290824cb3b9b9d609a81ec360e27939a7c9
Author: fred3m <fredem@princeton.edu>
Date:   Mon May 13 14:44:30 2019 -0500

    Update with scarlet master

commit afa31966330fd7c6ac77b2bcc522de69a76ca6da
Author: Nate Lust <nlust@astro.princeton.edu>
Date:   Mon May 13 12:27:35 2019 -0500

    Properly pass arguments to Source init
```

Extract lines like
```
grep "Unable to deblend" tiny4.o154140 >> UnableToDeblendLines
grep "Unable to deblend" tiny3.o154141 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154139 >> UnableToDeblendLines
grep "Unable to deblend" tiny4.o154138 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154137 >> UnableToDeblendLines
grep "Unable to deblend" tiny6.o154135 >> UnableToDeblendLines
grep "Unable to deblend" tiny5.o154131 >> UnableToDeblendLines
grep "Unable to deblend" tiny4.o154130 >> UnableToDeblendLines
grep "Unable to deblend" tiny3.o154129 >> UnableToDeblendLines

grep "meas.deblender.deblend: id:" tiny4.o154140 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny3.o154141 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154139 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny4.o154138 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154137 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny6.o154135 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny5.o154131 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny4.o154130 >> DeblendAttemptLines
grep "meas.deblender.deblend: id:" tiny3.o154129 >> DeblendAttemptLines
```

Note some of those logs are restarted failed reruns and the same deblend may appear more than once. I'm assuming double counting is fine in the context of rates.


## Results:

In [1]:
import pandas as pd
import os

DATA_DIR = '/datasets/hsc/repo/rerun/private/yusra/deblender-sprint/scarlet/w_2019_18_05k_noSym/logs'

df = pd.DataFrame.from_csv(os.path.join(DATA_DIR, 'UnableToDeblendLines'),  sep=': ', header=None)
attempted = pd.DataFrame.from_csv(os.path.join(DATA_DIR, 'DeblendAttemptLines'),  sep=': ', header=None)

/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/pandas/core/frame.py:1901: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  infer_datetime_format=infer_datetime_format)
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments 

In [2]:
groups = df.groupby(df[2])
groups.count()

,1
2,
'LsstSource' object has no attribute 'B',74
attempt to get argmax of an empty sequence,57
"could not broadcast input array from shape (41,41) into shape (41)",425
"could not broadcast input array from shape (41,43) into shape (41)",452
"could not broadcast input array from shape (43,41) into shape (43)",428
"could not broadcast input array from shape (43,43) into shape (43)",1813
tuple index out of range,3119


In [3]:
print("total attempted:", len(attempted))
print("number with exceptsion:", len(df[2]))
print("fraction with exceptions:", len(df[2])/len(attempted))

total attempted: 50562
number with exceptsion: 6384
fraction with exceptions: 0.12626082829002017


## Prototypes of each failure:

# Selections from tiny5.o154139: #
161845 INFO  2019-05-14T10:04:17.068-0500 multiBandDriver.deblendCoaddSources: Read 195 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '26,28'}, tag=set())
161845 INFO  2019-05-14T10:04:04.366-0500 multiBandDriver: lsst-verify-worker01:161845: Start merge detections from [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '24,17'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '24,17'}, tag=set())]
161845 INFO  2019-05-14T10:25:05.731-0500 meas.deblender.deblend: id: 43161724085338251
161845 WARN  2019-05-14T10:25:06.021-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161724085338251: could not broadcast input array from shape (43,43) into shape (43)
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 213, in computePsfImage
    psfs = np.array(psfs)
ValueError: could not broadcast input array from shape (43,43) into shape (43)

Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 213, in computePsfImage
    psfs = np.array(psfs)



---------------------
194805 INFO  2019-05-14T10:04:14.259-0500 multiBandDriver: lsst-verify-worker05:194805: Start deblending [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '25,8'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '25,8'}, tag=set())]
194805 INFO  2019-05-14T10:58:31.467-0500 meas.deblender.deblend: id: 43161500747038965
194805 WARN  2019-05-14T10:58:32.048-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161500747038965: 'LsstSource' object has no attribute 'B'
--------------------------




80278 INFO  2019-05-14T10:04:17.516-0500 multiBandDriver.deblendCoaddSources: Read 223 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '27,9'}, tag=set())
80278 INFO  2019-05-14T10:04:17.516-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Deblending 223 sources in 6 exposure bands
80278 WARN  2019-05-14T10:04:18.012-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161779919912961: attempt to get argmax of an empty sequence

----------------------------
Selections from tiny4.o154140
---------------------------
42797 INFO  2019-05-14T10:46:10.123-0500 multiBandDriver: lsst-verify-worker23:42797: Start deblending [DataId(initialdata={'tract': 9813, 'filter': 'HSC-G', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-R', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-I', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Z', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'HSC-Y', 'patch': '22,18'}, tag=set()), DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '22,18'}, tag=set())]
42797 INFO  2019-05-14T10:46:12.141-0500 multiBandDriver.deblendCoaddSources: Read 255 detections: DataId(initialdata={'tract': 9813, 'filter': 'NB0921', 'patch': '22,18'}, tag=set())
42797 INFO  2019-05-14T10:46:12.154-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Deblending 255 sources in 6 exposure bands
42797 INFO  2019-05-14T10:48:30.233-0500 meas.deblender.deblend: id: 43161131379851349
42797 WARN  2019-05-14T10:48:30.536-0500 multiBandDriver.deblendCoaddSources.multiBandDeblend: Unable to deblend source 43161131379851349: tuple index out of range
/project/yusra/lsst_devel/autograd/_eupspkg/binary/autograd/master-g6a9ef8c4bf/lib/python/autograd-1.2-py3.7.egg/autograd/numpy/fft.py:101: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return anp.pad(x, pads, 'constant')[slices]
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/source.py", line 22, in __init__
    symmetric, monotonic, center_step, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 331, in __init__
    thresh, True, monotonic)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 122, in init_extended_source
    sed = get_pixel_sed(sky_coord, scene, observations)  # amplitude is in sed
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 46, in get_pixel_sed
    raise SourceInitError(msg.format(*sky_coord))
scarlet.source.SourceInitError: Zero or negative flux at y=87, x=47

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 91, in deblend
    for center in peaks
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 91, in <listcomp>
    for center in peaks
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/source.py", line 27, in __init__
    center_step, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/source.py", line 253, in __init__
    super().__init__(self, sed, morph, **component_kwargs)
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/component.py", line 85, in __init__
    self.B = sed.shape[0]
  File "/project/yusra/lsst_devel/scarlet/_eupspkg/binary/scarlet/parameters-g9a43799fd3/lib/python/scarlet-0.59a43799-py3.7-linux-x86_64.egg/scarlet/component.py", line 110, in shape
    return (self.B, self.Ny, self.Nx)
AttributeError: 'LsstSource' object has no attribute 'B'
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 334, in __init__
    super().__init__(filters, array, Image, bbox)
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/image/multiband.py", line 163, in __init__
    bbox = Box2I(Point2I(0, 0), Extent2I(array.shape[2], array.shape[1]))
IndexError: tuple index out of range
Traceback (most recent call last):
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 401, in deblend
    blend = deblend(mExposure, foot, self.log, self.config)
  File "/project/yusra/lsst_devel/meas_extensions_scarlet/python/lsst/meas/extensions/scarlet/deblend.py", line 78, in deblend
    psfs = mExposure.computePsfImage(footprint.getCentroid()).array
  File "/software/lsstsw/stack_20190330/stack/miniconda3-4.5.12-1172c30/Linux64/afw/17.0.1-24-g0055ebd3c/python/lsst/afw/image/exposure/multiband.py", line 214, in computePsfImage
    psfImage = MultibandImage(self.filters, array=psfs)
